# Алгоритм Кнута-Морриса-Пратта

Алгоритм, позволяющий найти вхождение подстроки в строку.

Алгоритм прямого поиска имеет временную сложность O(n*m).<br>
Алгоритм Кнута-Морриса-Пратта в свою очередь имеет сложность O(n+m), что делает его более привлекательным для использования.<br>
Данный алгоритм применяется для решения следующих задач:
1. Анализ вхождений последовательности в структуру ДНК
2. Поиск слов в текстовых редакторах и поисковых системах
3. Обнаружение сигнатур вредоносного ПО

Алгоритм состоит из двух основных шагов:
1. Получение массива $π$ с помощью префикс-функции
2. Поиск вхождения подстроки в строку с помощью массива $π$

Данный алгоритм многим обязан так называемой "Префикс-функции", поэтому рассмотрение работы алгоритма начнется с неё.

## Префикс-функция

### Теория

Префикс функция используется для решения задач, связанных с нахождением повторяющихся последовательностей символов:
1. Сжатие данных
2. Нахождение повторяющихся цепочек ДНК
3. Нахождение повторяющихся слов в тексте

Пусть дана следующая строка: __abcabcd__ __(1)__

Префикс-функция использует такие понятия, как "суффикс" и "префикс": <br>
_Префикс_ - последовательность символов с начала рассматриваемой строки __(1)__: a, ab, abc, abca, abcab, abcabc <br>
_Суффикс_ - последовательность символов с конца рассматриваемой строки __(1)__: d, cd, bcd, abcd, cabcd, bcabcd <br>
Они не должны содержать все символы строки, поскольку это не будет иметь смысла. <br>

Префикс и суффикс соответственно имеют свою длину, равную количеству символов. <br>
Далее будем рассматривать префиксы и суффиксы равной длины для подстроки данной строки. <br>

Для подстроки "ab" имеется только один префикс и суффикс: 'a' и 'b' соответственно. <br>
Для подстроки "abc" имеется несколько суффиксов и префиксов: 'a' и 'b', "ab" и "bc" соответственно. <br>
И так далее...

Префикс функция для i-го символа возвращает `максимальную` длину совпавших префикса и суффикса подстроки из __(1)__, заканчивающейся на i-ый символ. <br>
Значения, полученные префикс-функцией для каждого элемента строки образуют массив $π$.

Для наглядности рассмотрим пример формирования массива $π$ для каждой подстроки из __(1)__:<br>
1. i=0, Для подстроки 'a' не сущетсвует префикса и суффикса, длина которого не равнялась бы длине самой подстроки. Вывод: 0.
2. i=1, Для подстроки "ab" cуществует только одна пара префикс-суфикс: 'a' и 'b', но они не совпадают. Вывод: 0.
3. i=2, Для подстроки "abc" cуществует две пары: 'a' и 'b', "ab" и "bc", но все они не совпадают. Вывод: 0.
4. i=3, Подстрока "abca" содержит совпадающую пару 'a' и 'a' (Несовпадающие: "ab" и "ca", "abc" и "bca"). Вывод: 1.
5. i=4, Подстрока "abcab" содержит совпадающую пару "ab" и "ab" (Несовпадающие: 'a' и 'b', "abc" и "cab", "abca" и "bcab"). Вывод: 2.
6. i=5, Подстрока "abcabc" содержит совпадающую пару "abc" и "abc". Вывод: 3.
7. i=6, Подстрока "abcabcd" не содержит ни одной совпадающей пары. Вывод: 0.

Массив $π$ будет содержать следующие значения: [0, 0, 0, 1, 2, 3, 0]



### Реализация алгоритма:

In [1]:
import pandas as pd
string = "abcabcd"

def prefix_func(pattern):
    m = len(pattern)
    pi = [0] * m
    k = 0

    for q in range(1, m):
        while k > 0 and pattern[k] != pattern[q]:
            k = pi[k - 1]
        if pattern[k] == pattern[q]:
            k += 1
        pi[q] = k

    return pi

pi = prefix_func(string)

df = pd.DataFrame({
    'Элемент': list(string),
    'Pi': pi
})

print(pi, end='\n\n')
df

[0, 0, 0, 1, 2, 3, 0]



,Элемент,Pi
0,a,0
1,b,0
2,c,0
3,a,1
4,b,2
5,c,3
6,d,0


## Поиск вхождения подстроки с помощью массива $π$

### Теория

Пусть дана строка **a = abcabcd** и посчитанный для неё **массив π**, а также строка  **b = abcabeabcabcabcd** <br>
| **i** | 0 | 1 | 2 | 3 | 4 | 5 | 6 |
| --- | --- | --- | --- | --- | --- | --- | --- |
| **a[i]** | a | b | c | a | b | c | d |
| **π[i]** | 0| 0 | 0 | 1 | 2 | 3 | 0 |

<br>

| **i** | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 | 11 | 12 | 13 | 14 | 15 |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| **b[i]** | a| b | c | a | b | e | a | b | c | a | b | c | a | b | c | d |

<br>



Найдем вхождения **a** в **b** с помощью массива $π$, полученного префикс-функцией: <br>
Начнём посимвольное сравнение:
| **i** | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 | 11 | 12 | 13 | 14 | 15 |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| **a[i]** | a | b | c | a | b | **c** | d |
| **b[i]** | a| b | c | a | b | **e** | a | b | c | a | b | c | a | b | c | d |

<br>

Дойдя до первого символа из **b**, который не соответствует таковому из **a**, мы обращаемся к массиву $π$: <br>

Из массива $π$ получаем значение элемента, индекс которого равен индексу последнего совпадающего символа из **a** (a[4]). Сдвигаем **a** относительно **b**  так, чтобы **a[π[4]] = 'c'** оказался на месте несовпадающего символа из **b**: <br>
| **i** | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 | 11 | 12 | 13 | 14 | 15 |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| **a[i]** | | | | a | b | **c** | a | b | c | d |
| **b[i]** | a| b | c | a | b | **e** | a | b | c | a | b | c | a | b | c | d |

<br>


Повторяем предыдущий шаг. Получаем элемент **a[π[2]] = 'a'**
| **i** | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 | 11 | 12 | 13 | 14 | 15 |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| **a[i]** | | | | | | **a** | b | c | a | b | c | d |
| **b[i]** | a| b | c | a | b | **e** | a | b | c | a | b | c | a | b | c | d |

<br>

В этот раз ситуация немного другая, первый же символ из **a** не совпадает с символом из **b**. В таком случае сдвигаем **a** на один элемент вправо.
| **i** | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 | 11 | 12 | 13 | 14 | 15 |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| **a[i]** | | | | | | | a | b | c | a | b | c | **d** |
| **b[i]** | a| b | c | a | b | e | a | b | c | a | b | c | **a** | b | c | d |

<br>

Повторяем алгоритм до тех пор, пока не будет найдено вхождение или не кончится строка. Сдвигаем **a[π[5]] = 'a'** на место несовпавшего элемента.
| **i** | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 | 11 | 12 | 13 | 14 | 15 |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| **a[i]** | | | | | | | | | | **a** | **b** | **c** | **a** | **b** | **c** | **d** |
| **b[i]** | a| b | c | a | b | e | a | b | c | **a** | **b** | **c** | **a** | **b** | **c** | **d** |

Вхождение **a** в **b** найдено при i=9.

### Реализация алгоритма:

In [2]:
substr = "abcabcd"
string = "abcabeabcabcabcd"

def kmp(s, t):
    index = -1
    f = prefix_func(s)
    k = 0
    for i in range(len(t)):
        while k > 0 and s[k] != t[i]:
            k = f[k-1]
        if s[k] == t[i]:
            k = k + 1
        if k == len(s):
            index = i - len(s) + 1
            break
    return index

index = kmp(substr, string)

if (index > 0):
    print ("Вхождение найдено, индекс:", index)
else:
    print("Вхождение не найдено")

Вхождение найдено, индекс: 9


## Решение прикладной задачи

Найти индекс вхождения выражения "Уйдет зима" в стихе "Меняются и время и мечты".

In [3]:
substr = "Уйдет зима"
string = """Меняются и время, и мечты; 
Меняются, как время, представленья. 
Изменчивы под солнцем все явленья, 
И мир всечасно видишь новым ты. 
Во всем и всюду новые черты, 
Но для надежды нет осуществленья. 
От счастья остаются сожаленья, 
От горя — только чувство пустоты. 
Уйдет зима, уйдут снега и холод, 
И мир весной, как прежде, станет молод, 
Но есть закон: все обратится в тлен. 
Само веселье слез не уничтожит, 
И страшно то, что час пробьет, быть может, 
Когда не станет в мире перемен."""

index = kmp(substr, string)
if (index > 0):
    print ("Вхождение найдено, индекс:", index)
else:
    print("Вхождение не найдено")

Вхождение найдено, индекс: 266
